## Lab 2 - Lemmatization and full text search

In [1]:
import os
from elasticsearch import Elasticsearch

### Tasks 1 & 2
Installing elasticsearch and plugins.  
Elasticsearch must be running in the background.

In [2]:
es = Elasticsearch()
es.cat.plugins(params={'h': 'component,version,description'})

'analysis-morfologik 7.9.2 Morfologik Polish Lemmatizer plugin for Elasticsearch\n'

### Task 3
Lowercase filter was available by default, Morfologik-based lemmatizer is from the plugin `morfologic_stem`, and for synonyms - filter of that type was created.

In [3]:
index_config = {
    "analysis": {
        "filter": {
            "legislative_synonym_filter": {
                "type":"synonym",
                "synonyms": [
                    "kpk => kodeks postępowania karnego",
                    "kpc => kodeks postępowania cywilnego",
                    "kk => kodeks karny",
                    "kc => kodeks cywilny"
                ]
            }
        },
        "analyzer": {
            "default": {
                "type":"custom",
                "tokenizer":"standard",
                "filter": [
                    "lowercase", 
                    "legislative_synonym_filter", 
                    "morfologik_stem"
                ]
            }
        }
    }
}

### Task 4

In [4]:
index_name = "polish_bills"

if es.indices.exists(index=index_name): # in case of multiple runs, delete if index already exists
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body={"settings": index_config})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'polish_bills'}

### Task 5

In [5]:
bills_dir = "../bills/"
bills_files = os.listdir(bills_dir)

for file_path in [bills_dir + filename for filename in  bills_files]:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    es.index(index=index_name, body={"filename": file_path[len(bills_dir):], "text": text})

Check if all files were added:

In [6]:
print(len(es.search(index=index_name, size=1500)['hits']['hits']) == len(bills_files))

True


### Task 6

https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html

In [7]:
match_query_1 = es.search(index=index_name, body={"query": {"match": {"text": "ustawa"}}})
print("There is {} acts with word 'ustawa' in them.".format(match_query_1["hits"]["total"]["value"]))

There is 1179 acts with word 'ustawa' in them.


There is 1180 files so only one file didn't have a word `ustawa` in it.

### Task 7
https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html

In [8]:
match_query_2 = es.search(index=index_name, body={"query": {"match_phrase": {"text": "kodeks postępowania cywilnego"}}})
print("There is {} acts with phrase 'kodeks postępowania cywilnego' in any form."
      .format(match_query_2["hits"]["total"]["value"]))

There is 100 acts with phrase 'kodeks postępowania cywilnego' in any form.


### Task 8
The slop parameter allows additional terms in the searched phrase (in this case, slop=2 allows for up to two additional words in phrase `wchodzi w życie`). 

In [9]:
query = {
    "query": {
        "match_phrase": {
            "text": {
                "query": "wchodzi w życie",
                "slop": 2
            }
        }
    }
}

match_query_3 = es.search(index=index_name, body=query)
print("Number of acts with phrase 'wchodzi w życie' (and slop=2): {}".format(match_query_3["hits"]["total"]["value"]))

Number of acts with phrase 'wchodzi w życie' (and slop=2): 1175


### Task 9

Results are already sorted by relevance (parameter `_score`):

In [10]:
match_query_4 = es.search(index=index_name, body={"query": {"match": {"text": "konstytucja"}}, "size": 10})
# not displaying the file itself for readability
print([{k: v for k, v in res.items() if k != "_source"} for res in match_query_4["hits"]["hits"][:3]])

[{'_index': 'polish_bills', '_type': '_doc', '_id': '0QG7SnUBBtupf1htoz1V', '_score': 6.8715763}, {'_index': 'polish_bills', '_type': '_doc', '_id': '0AG7SnUBBtupf1htxz7P', '_score': 6.6677475}, {'_index': 'polish_bills', '_type': '_doc', '_id': 'zwG7SnUBBtupf1htoz0f', '_score': 6.63778}]


Top 10 most relevant acts for the phrase `konstytucja`: 

In [11]:
most_relevant = [m["_source"]["filename"] for m in match_query_4["hits"]["hits"]]
print(most_relevant)

['1997_629.txt', '2000_443.txt', '1997_604.txt', '1996_350.txt', '1997_642.txt', '2001_23.txt', '1996_199.txt', '1999_688.txt', '1997_681.txt', '2001_1082.txt']


### Task 10

https://www.elastic.co/guide/en/elasticsearch/reference/current/highlighting.html

In [12]:
query = {
    "query": {
        "match": {
            "text": "konstytucja"
        },
    },
    "size" : 10, # top 10 acts
    "highlight": {
        "fields": {
            "text": {} # highlighting content of a bill, no additional settings
        },
        "number_of_fragments": 3 # number of excerpts
    }
}
    
match_query_5 = es.search(index=index_name, body=query)
highlights = [{"filename": res["_source"]["filename"], "highlights": res["highlight"]["text"]} 
              for res in match_query_5["hits"]["hits"]]

for h in highlights:
    print("\n{}:\n\t{})".format(h["filename"], "\n\t".join(h["highlights"])))


1997_629.txt:
	o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
	W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
	Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie)

2000_443.txt:
	umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
	międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
	co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>)

1997_604.txt:
	Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
	Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalności
	Ciężar udowodnienia 